In [1]:
import nltk

In [2]:
def pos_feature(sentence, i, history):
    feature = {
        "suffix(1)": sentence[i][-1:],
        "suffix(2)": sentence[i][-2:],
        "suffix(3)": sentence[i][-3:]
    }
    if i == 0:
        feature["prev-word"] = "<START>"
        feature["prev-tag"] = "<START>"
    else:
        feature["prev-word"] = sentence[i-1]
        feature["prev-tag"] = history[i-1]
    return feature

In [4]:
class ConsecutivePosTagger(nltk.TaggerI):
    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = pos_feature(untagged_sent, i, history)
                train_set.append((featureset, tag))
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)
        
    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = pos_feature(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

In [5]:
tagged_sents = nltk.corpus.brown.tagged_sents(categories='news')

In [6]:
size = int(len(tagged_sents)*0.1)

In [7]:
train_sents, test_sents = tagged_sents[size:], tagged_sents[:size]

In [8]:
tagger = ConsecutivePosTagger(train_sents)

In [9]:
tagger.evaluate(test_sents)

0.7980528511821975